###**Model C (transfer, inc_res_v2) HYPERPARAMETERS** _(~94% on test set)_**

> SEED: 1234

> train_dataset: 80%

> valid_dataset: 20%

> Batch size: 32

> Image w/h: 299

> Data Augmentation : Custom

> CNN structure:
>> Inception Resnet V2
>
>> All Trainable

> FCN structure:
>> Global Average Pooling layer
>
>> Dense 1024 + HE init
>
>> Batch Normalization (Standard)
>
>> Activation : LeakyReLu
>
>> Dropout (0.5, seed= SEED)
>
>> Dense 256 + HE init
>
>> Batch Normalization (Standard)
>
>> Activation : LeakyReLu
>
>> Dropout (0.5, seed= SEED)
>
>> 3 neurons (Softmax)

> Training parameters:
>> Optimizer: Adam
>
>> Epochs: 50
>
>> Early stopping: 10 epochs
>
>> Learning rate: 1e-3
>
>> Loss: Categorical crossentropy
>
>> ReduceLROnPlateau
>
>> Test Time Augmentation

In [ ]:
# Helper libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from datetime import datetime
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_resnet_v2 import preprocess_input

# Fixed a seed to make results reproducible 
SEED = 1234
tf.random.set_seed(SEED)

# Getting current main directory
cwd = os.getcwd()

In [ ]:
# Mounting G Drive folder
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# CONSTANTS
bs = 32
img_h = 299
img_w = 299
num_classes = 3

In [ ]:
dataset_dir = os.path.join(cwd, '/content/drive/My Drive/ANN/Dataset')
training_dir = os.path.join(dataset_dir, 'training')
valid_dir = os.path.join(dataset_dir, 'validation')
test_dir = os.path.join(dataset_dir, 'test')

In [ ]:
# Data augmentation
apply_data_augmentation = True

# Creating training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=20,
                                        width_shift_range=20,
                                        height_shift_range=20,
                                        channel_shift_range=20,
                                        zoom_range=[0.8,1.2],
                                        shear_range=10,
                                        horizontal_flip=True,
                                        vertical_flip=False,
                                        brightness_range=[0.6,1.4],
                                        fill_mode='nearest',
                                        preprocessing_function=preprocess_input )
else:
    train_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

valid_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_data_gen = ImageDataGenerator(rotation_range=20,
                                        width_shift_range=20,
                                        height_shift_range=20,
                                        channel_shift_range=20,
                                        zoom_range=[0.8,1.2],
                                        shear_range=10,
                                        horizontal_flip=True,
                                        vertical_flip=False,
                                        brightness_range=[0.6,1.4],
                                        fill_mode='nearest',
                                        preprocessing_function=preprocess_input )

In [ ]:
# Taking the path to a directory and generating batches of augmented data
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               target_size=(img_h, img_w),
                                               color_mode="rgb",
                                               batch_size=bs, 
                                               class_mode='categorical',
                                               classes=['NO_MASK','ALL_MASK','SOME_MASK'],
                                               shuffle=True,
                                               seed=SEED)

valid_gen = valid_data_gen.flow_from_directory(valid_dir,
                                               target_size=(img_h, img_w),
                                               color_mode="rgb",
                                               batch_size=bs, 
                                               class_mode='categorical',
                                               classes=['NO_MASK','ALL_MASK','SOME_MASK'],
                                               shuffle=False,
                                               seed=SEED)

test_gen = test_data_gen.flow_from_directory(test_dir,
                                               target_size=(img_h, img_w),
                                               color_mode="rgb",
                                               batch_size=1, 
                                               class_mode='categorical',
                                               classes=None,
                                               shuffle=False,
                                               seed=SEED)

In [ ]:
# Transfer learning
inc_res = tf.keras.applications.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

finetuning = True

if finetuning:
    inc_res.trainable = True
else:
    inc_res.trainable = False

In [ ]:
#Model
model = tf.keras.Sequential()
model.add(inc_res)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(units=1024, kernel_initializer='he_uniform'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.Dropout(0.5, seed=SEED))
model.add(tf.keras.layers.Dense(units=256, kernel_initializer='he_uniform'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.Dropout(0.5, seed=SEED))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [ ]:
# Optimization params
loss = tf.keras.losses.CategoricalCrossentropy()
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
metrics = ['accuracy']
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# Visualize created model as a table
model.summary()
for i, layer in enumerate(inc_res.layers):
  print(i, layer.name, "-", layer.trainable)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir /content/drive/My\ Drive/ANN/Log

In [ ]:
# Callbacks

cwd = '/content/drive/My Drive/ANN'

exps_dir = os.path.join(cwd, 'Log')

if not os.path.exists(exps_dir):
  os.makedirs(exps_dir)


now = datetime.now().strftime('%b%d_%H-%M-%S') # taking instant time
exp_name = 'FC'                                # name of experiment
exp_dir = os.path.join(exps_dir, exp_name + '_' + str(now))


if not os.path.exists(exp_dir):
  os.makedirs(exp_dir)


callbacks = [] 

ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
  os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), save_weights_only=True, save_best_only=True)

callbacks.append(ckpt_callback)

tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
  os.makedirs(tb_dir)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1, cooldown=0)
callbacks.append(reduce_lr)

tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, histogram_freq=1)
callbacks.append(tb_callback)

es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
callbacks.append(es_callback)

In [ ]:
### training the model ###
model.fit(
    x=train_gen,
    y=None,
    epochs=70,
    callbacks=callbacks,
    steps_per_epoch=len(train_gen),
    validation_data=valid_gen,
    validation_steps=len(valid_gen)
)

In [ ]:
def create_csv(results, results_dir='./'):

    csv_fname = 'Last'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
# Save the model
model.save('/content/drive/My Drive/ANN/Models/Last')

In [ ]:
#Test Time Augmentation and CSV generation
from tqdm import tqdm
from datetime import datetime
test_gen.reset()
tta_steps = 10 #number of time to perform TTA
predictions = []
results = {}

for i in tqdm(range(tta_steps)):
    preds = model.predict(test_gen,
                          batch_size = bs,
                          verbose=1)
    predictions.append(preds)

pred = np.mean(predictions, axis = 0)
predicted_class_indices = np.argmax(pred, axis=-1)
filenames = test_gen.filenames
prova = [e[7:] for e in filenames]

for i in range(0, len(test_gen)):
  results[prova[i]] = predicted_class_indices[i]

create_csv(results=results, results_dir=test_dir)